In [ ]:
!pip install ktrain
!pip install mpi4py

     |████████████████████████████████| 25.3 MB 59.5 MB/s 
     |████████████████████████████████| 6.8 MB 21.2 MB/s 
     |████████████████████████████████| 981 kB 42.4 MB/s 
     |████████████████████████████████| 263 kB 48.7 MB/s 
     |████████████████████████████████| 2.8 MB 39.6 MB/s 
     |████████████████████████████████| 1.2 MB 38.5 MB/s 
     |████████████████████████████████| 468 kB 21.9 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 3.3 MB 40.5 MB/s 
     |████████████████████████████████| 596 kB 48.4 MB/s 
     |████████████████████████████████| 895 kB 45.0 MB/s 
  Created wheel for ktrain: filename=ktrain-0.28.3-py3-none-any.whl size=25292659 sha256=e7503e6ad1bd83e106dcc06ec5e891c3c94d214804054094a37befcec0c6f9df
  Stored in directory: /root/.cache/pip/wheels/6a/7e/c3/f46cdfc2b81c54424923b1405d7e670c35cacc11ada9a47b1c
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=b0c1c21b1af1b77

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [ ]:
# df = pd.read_csv('/content/consolidated.csv')
df = pd.read_csv('drive/MyDrive/Datasets/CSE4001_PDC_Project/consolidated.csv')

In [ ]:
df

,Date,1,2,3,4,5,6,7,8,9,10,11,12,Label,Pct_inc,Bin_label,Up,Down
0,08-09-1998,Boeing gets Varig order,Airbus gets Emirates order,Dow set for early takeoff,CNNfn market movers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inc,1.936619718,Up,1,0
1,09-09-1998,UPS flies with Airbus,Boeing Airbus split order,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dec,-2.590673575,Down,0,1
2,05-10-1998,Boeing boosts output,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inc,2.111324376,Up,1,0
3,07-10-1998,Raytheon cuts 14000 jobs,Open skies' at an impasse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dec,-2.466793169,Down,0,1
4,08-10-1998,Wall Street struggles in pain,Selling pressure eases,Wall Street reels in pain,Wall St. shaken up,More turmoil for U.S. stocks,Yen takes second bite of bond,NaN,NaN,NaN,NaN,NaN,NaN,Dec,-2.946954813,Down,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6073,17-02-2017,Trump and jobs: A running fact check,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inc,0.784543054,Up,1,0
6074,21-02-2017,Move over Amazon: Here comes Walmart,Retailers report; Central bank chatter; Stock ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inc,0.337028512,Up,1,0
6075,22-02-2017,These 10 stocks dominate the market,How Trump could make the trade deficit look worse,Amazon brings free shipping minimum back down ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dec,-0.747406228,Down,0,1
6076,23-02-2017,Fallen angels? Victoria's Secret sales plunge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dec,-0.958336111,Down,0,1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
df.head()

,Date,1,2,3,4,5,6,7,8,9,10,11,12,Label,Pct_inc,Bin_label,Up,Down
0,08-09-1998,Boeing gets Varig order,Airbus gets Emirates order,Dow set for early takeoff,CNNfn market movers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inc,1.936619718,Up,1,0
1,09-09-1998,UPS flies with Airbus,Boeing Airbus split order,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dec,-2.590673575,Down,0,1
2,05-10-1998,Boeing boosts output,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inc,2.111324376,Up,1,0
3,07-10-1998,Raytheon cuts 14000 jobs,Open skies' at an impasse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dec,-2.466793169,Down,0,1
4,08-10-1998,Wall Street struggles in pain,Selling pressure eases,Wall Street reels in pain,Wall St. shaken up,More turmoil for U.S. stocks,Yen takes second bite of bond,NaN,NaN,NaN,NaN,NaN,NaN,Dec,-2.946954813,Down,0,1


In [ ]:
df2 = df[['1','2','3','4','5','6','7','8','9','10','11','12','Bin_label']]
df2["News Headline"] = ""
for i in range(len(df2)):
  for j in df2.loc[i]:
    if not pd.isna(j):
      df2['News Headline'][i] = df2['News Headline'][i]+". "+str(j)
    else:
      break
df3 = df2[['News Headline','Bin_label']]
df3.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,News Headline,Bin_label
0,. Boeing gets Varig order. Airbus gets Emirate...,Up
1,. UPS flies with Airbus. Boeing Airbus split ...,Down
2,. Boeing boosts output,Up
3,. Raytheon cuts 14000 jobs. Open skies' at an ...,Down
4,. Wall Street struggles in pain. Selling press...,Down


In [ ]:
df3.to_csv('con_3.csv')

In [ ]:
df3['News Headline'][4910]

". A surprise leap in jobs wages. Thursday's tech rally"

In [ ]:
x = df2['News Headline']
y= df3['Bin_label']
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=32)

In [ ]:
df['Bin_label'].unique

<bound method Series.unique of 0         Up
1       Down
2         Up
3       Down
4       Down
        ... 
6073      Up
6074      Up
6075    Down
6076    Down
6077      Up
Name: Bin_label, Length: 6078, dtype: object>

In [ ]:
(X_train,y_train),(X_test,y_test),preprocess = text.texts_from_df(train_df = df3,
                                                                  text_column = 'News Headline',
                                                                  label_columns = 'Bin_label',
                                                                  val_df = df3,
                                                                  maxlen = 30,
                                                                  preprocess_mode = 'bert')

['Bin_label', 'Down', 'Up']
   Bin_label  Down   Up
0        0.0   0.0  1.0
1        0.0   1.0  0.0
2        0.0   0.0  1.0
3        0.0   1.0  0.0
4        0.0   1.0  0.0
['Bin_label', 'Down', 'Up']
   Bin_label  Down   Up
0        0.0   0.0  1.0
1        0.0   1.0  0.0
2        0.0   0.0  1.0
3        0.0   1.0  0.0
4        0.0   1.0  0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
df3['Bin_label'].values

array(['Up', 'Down', 'Up', ..., 'Down', 'Down', 'Up'], dtype=object)

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0,1";  

import tensorflow as tf
mirrored_strategy = tf.distribute.MirroredStrategy()
BATCH_SIZE = 6 * 2 # desired BS times 2
MODEL_NAME = 'bert'
# t = text.Transformer(MODEL_NAME, maxlen=500, class_names=df3['Bin_label'].values)
# trn = t.preprocess_train(x_train, y_train)
# val = t.preprocess_test(x_test, y_test)
with mirrored_strategy.scope():
    model = text.text_classifier(name = 'bert',
                             train_data = (X_train,y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 30
done.


In [ ]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train,y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 30
done.


In [ ]:
# learner = ktrain.get_learner(model = model,
#                              train_data = (X_train,y_train),
#                              val_data = (X_test,y_test),
#                              batch_size = 6)
learner = ktrain.get_learner(model = model,
                             train_data = (X_train,y_train),
                             val_data = (X_test,y_test),
                             batch_size = BATCH_SIZE)

In [ ]:
learner.fit_onecycle(lr = 1e-5,epochs = 10)



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/10
507/507 [==============================] - 227s 399ms/step - loss: 0.7557 - accuracy: 0.5018 - val_loss: 0.6976 - val_accuracy: 0.5188
Epoch 2/10
507/507 [==============================] - 197s 389ms/step - loss: 0.7017 - accuracy: 0.5064 - val_loss: 0.6889 - val_accuracy: 0.5318
Epoch 3/10
507/507 [==============================] - 197s 389ms/step - loss: 0.6957 - accuracy: 0.5209 - val_loss: 0.6704 - val_accuracy: 0.5969
Epoch 4/10
507/507 [==============================] - 197s 389ms/step - loss: 0.6791 - accuracy: 0.5729 - val_loss: 0.6021 - val_accuracy: 0.7415
Epoch 5/10
507/507 [==============================] - 197s 389ms/step - loss: 0.6318 - accuracy: 0.6491 - val_loss: 0.4535 - val_accuracy: 0.8254
Epoch 6/10
507/507 [==============================] - 197s 389ms/step - loss: 0.4682 - accuracy: 0.7836 - val_loss: 0.1849 - val_accuracy: 0.9589
Epoch 7/10
507/507 [==============================] - 197s 38

In [ ]:
# learner.save_model('NLP_Model')
# learner.load_model('NLP_Model')
learner.save_model('drive/MyDrive/Models')
# learner.load_model('drive/MyDrive/Models')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
predictor = ktrain.get_predictor(learner.model,preprocess)

In [ ]:
data = ["Google defends growth plans. Google defends growth plans. Stocks end day higher but May is a loss. Stocks rebound ahead of Fed. Pining for a pause. U.S. asks Internet firms to keep customer info. Dec"	]

In [ ]:
predictor.predict(data)

['Down']

In [ ]:
# from joblib import dump

# # Save the model as a pickle in a file
# dump(learner, 'drive/MyDrive/Models/model_BERT.pkl')

In [ ]:
import time

start_time = time.time()
for i in range(len(df)):
  data = df3['News Headline'][i]
  pred = predictor.predict(data)
  print(df3['Bin_label'][i],pred)
end_time = time.time()
print("Execution Time : ", (end_time - start_time))

Streaming output truncated to the last 5000 lines.
Down Down
Up Up
Up Up
Up Up
Up Up
Down Down
Down Down
Up Up
Down Down
Up Up
Down Down
Down Down
Up Up
Up Up
Up Up
Up Up
Down Down
Up Up
Up Up
Down Down
Up Up
Down Down
Up Up
Down Down
Down Down
Up Up
Up Up
Down Down
Down Down
Down Down
Up Up
Down Down
Up Up
Up Up
Down Down
Down Down
Up Up
Down Down
Down Down
Up Up
Down Down
Up Up
Up Up
Up Up
Up Up
Up Up
Down Down
Up Up
Up Up
Down Down
Up Up
Down Down
Up Up
Up Up
Down Down
Down Down
Up Up
Up Up
Up Up
Up Up
Down Down
Down Down
Down Down
Down Down
Up Up
Down Down
Down Down
Up Up
Down Up
Down Down
Up Up
Up Up
Down Down
Down Down
Up Up
Down Down
Up Up
Up Up
Up Up
Down Down
Up Up
Down Down
Down Down
Up Up
Down Down
Up Up
Up Up
Up Up
Up Up
Down Down
Down Down
Down Down
Down Down
Up Up
Up Up
Up Up
Up Up
Down Down
Up Up
Up Up
Up Up
Up Up
Down Down
Down Down
Down Down
Up Up
Up Up
Down Down
Down Down
Down Down
Down Down
Up Up
Up Up
Up Up
Down Down
Down Down
Down Up
Up Up
Down Down
Up Up
Down Down

In [ ]:
df3 = pd.read_csv('con_3.csv', index_col = 0)
df3

,News Headline,Bin_label
0,. Boeing gets Varig order. Airbus gets Emirate...,Up
1,. UPS flies with Airbus. Boeing Airbus split ...,Down
2,. Boeing boosts output,Up
3,. Raytheon cuts 14000 jobs. Open skies' at an ...,Down
4,. Wall Street struggles in pain. Selling press...,Down
...,...,...
6073,. Trump and jobs: A running fact check,Up
6074,. Move over Amazon: Here comes Walmart. Retail...,Up
6075,. These 10 stocks dominate the market. How Tru...,Down
6076,. Fallen angels? Victoria's Secret sales plunge,Down


In [ ]:
%%writefile predict.py
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import mpi4py
import time


mpi4py.rc.initialize = False  # do not initialize MPI automatically
mpi4py.rc.finalize = False    # do not finalize MPI automatically

from mpi4py import MPI

MPI.Init()      # manual initialization of the MPI environment
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()


# df = pd.read_csv('drive/MyDrive/Datasets/CSE4001_PDC_Project/consolidated.csv')
# df2 = df[['1','2','3','4','5','6','7','8','9','10','11','12','Bin_label']]
# df2["News Headline"] = ""
# for i in range(len(df2)):
#   for j in df2.loc[i]:
#     if not pd.isna(j):
#       df2['News Headline'][i] = df2['News Headline'][i]+". "+str(j)
#     else:
#       break
df3 = pd.read_csv('con_3.csv', index_col = 0)

(X_train,y_train),(X_test,y_test),preprocess = text.texts_from_df(train_df = df3,
                                                                  text_column = 'News Headline',
                                                                  label_columns = 'Bin_label',
                                                                  val_df = df3,
                                                                  maxlen = 30,
                                                                  preprocess_mode = 'bert')

# import tensorflow as tf
# mirrored_strategy = tf.distribute.MirroredStrategy()
# BATCH_SIZE = 6 * 2 # desired BS times 2
# MODEL_NAME = 'bert'
# t = text.Transformer(MODEL_NAME, maxlen=500, class_names=df3['Bin_label'].values)
# trn = t.preprocess_train(x_train, y_train)
# val = t.preprocess_test(x_test, y_test)
# with mirrored_strategy.scope():
#     model = text.text_classifier(name = 'bert',
#                              train_data = (X_train,y_train),
#                              preproc = preprocess)

model = text.text_classifier(name = 'bert',
                             train_data = (X_train,y_train),
                             preproc = preprocess)
learner = ktrain.get_learner(model = model,
                             train_data = (X_train,y_train),
                             val_data = (X_test,y_test),
                             batch_size = 6)
learner.load_model('drive/MyDrive/Models')

predictor = ktrain.get_predictor(learner.model,preprocess)

# if rank == 0 :
#   p = len(df3)//size
#   data = df3['News Headline'].values
#   for i in range(1,size) :
#         comm.send(p , dest=i, tag=11)
#         comm.Send([data[(i-1)*p:i*p], MPI.CHAR], dest=i, tag=11)
#   if rank != 0:
#     p = comm.recv(source=0, tag=11)
#     # print(p)
#     data = np.empty(p, dtype='i')
#     comm.Recv([data, MPI.CHAR] ,source=0, tag=11)
#     # print(rank)
#     print(data)

# start_time = time.time()
p = len(df3)//size

if (rank != size-1) :
  for i in range(p*rank, p*(rank +1)) :
    data = df3['News Headline'][i]
    pred = predictor.predict(data)
    print("Processor : ", rank, end=" ")
    print(data, end="   ")
    print(df3['Bin_label'][i],pred)

if (rank == size - 1) :
  for i in range(p*rank, len(df3)) :
    data = df3['News Headline'][i]
    pred = predictor.predict(data)
    print("Processor : ", rank, end=" ")
    print(data, end="   ")
    print(df3['Bin_label'][i],pred)


# for i in range(10):
#   data = df3['News Headline'][i]
#   pred = predictor.predict(data)
#   print(df3['Bin_label'][i],pred)
MPI.Finalize()  # manual finalization of the MPI environment
# end_time = time.time()
# print("Execution Time : ", (end_time - start_time))

Overwriting predict.py


In [ ]:
!mpiexec -n 4 --allow-run-as-root python predict.py

['Bin_label', 'Down', 'Up']
['Bin_label', 'Down', 'Up']
['Bin_label', 'Down', 'Up']
['Bin_label', 'Down', 'Up']
   Bin_label  ...   Up
0        0.0  ...  1.0
1        0.0  ...  0.0
2        0.0  ...  1.0
3        0.0  ...  0.0
4        0.0  ...  0.0

[5 rows x 3 columns]
   Bin_label  ...   Up
0        0.0  ...  1.0
1        0.0  ...  0.0
2        0.0  ...  1.0
3        0.0  ...  0.0
4        0.0  ...  0.0

[5 rows x 3 columns]   Bin_label  ...   Up
0        0.0  ...  1.0
1        0.0  ...  0.0
2        0.0  ...  1.0
3        0.0  ...  0.0
4        0.0  ...  0.0

[5 rows x 3 columns]

   Bin_label  ...   Up
0        0.0  ...  1.0
1        0.0  ...  0.0
2        0.0  ...  1.0
3        0.0  ...  0.0
4        0.0  ...  0.0

[5 rows x 3 columns]
['Bin_label', 'Down', 'Up']
['Bin_label', 'Down', 'Up']
['Bin_label', 'Down', 'Up']
['Bin_label', 'Down', 'Up']
   Bin_label  ...   Up
0        0.0  ...  1.0
1        0.0  ...  0.0
2        0.0  ...  1.0
3        0.0  ...  0.0
4        0.0  ...  0.